# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
CLIENT_PORT = 62012
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", CLIENT_PORT)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

Sun 2023-01-22 10:48:39.235552 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE | WRITABLE
Sun 2023-01-22 10:48:39.236415 [INFO ] - [T] Attempting to bind to 0.0.0.0:62012
Sun 2023-01-22 10:48:39.236435 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
Sun 2023-01-22 10:48:39.237519 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
Sun 2023-01-22 10:48:39.271758 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7fafcdede950>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
CRESET    = '\33[0m'
CBOLD     = '\33[1m'
CITALIC   = '\33[3m'
CURL      = '\33[4m'
CBLINK    = '\33[5m'
CBLINK2   = '\33[6m'
CSELECTED = '\33[7m'

CBLACK  = '\33[30m'
CRED    = '\33[31m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CBLUE   = '\33[34m'
CVIOLET = '\33[35m'
CBEIGE  = '\33[36m'
CWHITE  = '\33[37m'

def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    else:
        color = CVIOLET
    print(f"{color}{message}{CRESET}")
        

class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
client_tiface

server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
server_tiface

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

(client_fiface, server_fiface)

(<FilterInterface at 0x7fafcc538c20>, <FilterInterface at 0x7fafcc538fb0>)

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command',
 'command_response',
 'get_notifications',
 'notif_poll_ms',
 'response',
 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

LTI(client)0: got transport notifications: []
LTI(server)0: got transport notifications: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation

### As a client, connect to server

In [10]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" }

In [11]:
jupyter_endpoint = EndpointW(f"127.0.0.1:{SERVER_PORT}")

filter_cmd = FilterCmdW("sendrequestaction", endpoint=jupyter_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }

In [12]:
await client_fiface.command_response(filter_cmd)

Sun 2023-01-22 10:48:39.456528 [TRACE] - [F<-A,C] New command: SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }
LTI(client)5 skipped 4: received transport_cmd NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 999999999s }
Sun 2023-01-22 10:48:39.456909 [TRACE] - [T<-F,C] Processing command NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 999999999s }
LTI(client)6: got transport_rsp back from Transport layer: TransportRsp::Accepted


Accepted

Sun 2023-01-22 10:48:39.458469 [TRACE] - [F<-T,R] Command Accepted


In [13]:
# XXX get endpoint programatically from client
client_endpoint = EndpointW(f"127.0.0.1:{CLIENT_PORT}")
client_endpoint

Endpoint(127.0.0.1:62012)

In [15]:
server_notifications = server_fiface.get_notifications()
assert len(server_notifications) > 0
new_request_action = server_notifications[0]

assert new_request_action.request_action == connect_action

Sun 2023-01-22 10:48:47.208596 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }
Sun 2023-01-22 10:48:47.220859 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }
Sun 2023-01-22 10:48:47.467808 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }
Sun 2023-01-22 10:48:47.468905 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }


In [16]:
cookie = base64.urlsafe_b64encode(bytes(new_request_action.request_action.name.encode()))
cookie

b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'

In [17]:
response_ok = ResponseCodeW("loggedin", cookie=str(cookie), server_version="0.0.666")
response_ok

Sun 2023-01-22 10:48:47.718476 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }
Sun 2023-01-22 10:48:47.748073 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }


ResponseCodeW { inner: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'", server_version: "0.0.666" } }

In [18]:
await server_fiface.command_response(FilterCmdW("sendresponsecode", endpoint=client_endpoint, code=response_ok))

Sun 2023-01-22 10:48:47.978476 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }
Sun 2023-01-22 10:48:47.995719 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }
LTI(server)287 skipped 276: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62012), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 0 }, retry_interval: 9999s }], packets: [Response { sequence: 1, request_ack: Some(1), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'", server_version: "0.0.666" } }] }
Sun 2023-01-22 10:48:48.166629 [TRACE] - [F<-A,C] New command: SendResponseCode { endpoint: Endpoint(127.0.0.1:62012), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'", server_version: "0.0.666" }

Accepted

Sun 2023-01-22 10:48:48.169971 [TRACE] - [F<-T,R] Command Accepted
LTI(server)289: got transport notifications: []
Sun 2023-01-22 10:48:48.238173 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }
Sun 2023-01-22 10:48:48.244339 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5089", client_version: "0.0.666" } }
Sun 2023-01-22 10:48:48.245426 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62013), Took packet Response { sequence: 1, request_ack: Some(1), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'", server_version: "0.0.666" } }
LTI(client)96 skipped 86: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 0 } }
Sun 2023-01-22 10:48:48.245715 [TRACE] - [T<-F,C] Processing comma

In [19]:
client_notifications = client_fiface.get_notifications()
assert len(client_notifications) > 0
new_response_code = client_notifications[0]
new_response_code

NewResponseCode { endpoint: Endpoint(127.0.0.1:62013), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'", server_version: "0.0.666" } }

In [20]:
new_response_code.response_code

ResponseCodeW { inner: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'", server_version: "0.0.666" } }

In [21]:
cookie_from_server = new_response_code.response_code.cookie
assert str(cookie_from_server) == str(cookie)

### Convenience function

In [22]:
async def request_action_to_filter(req_action, endpoint=jupyter_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await client_fiface.command_response(filter_cmd)

def get_all_notification(fiface, min_expected_count=1):
    notifications = fiface.get_notifications()
    assert len(notifications) >= min_expected_count
    
    return notifications

### Create a room

In [23]:
await request_action_to_filter(RequestActionW("newroom", room_name="general"))

Sun 2023-01-22 10:48:50.996950 [TRACE] - [F<-A,C] New command: SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: NewRoom { room_name: "general" } }
LTI(client)125 skipped 26: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }] }


Accepted

Sun 2023-01-22 10:48:50.997399 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }] }
Sun 2023-01-22 10:48:50.997452 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
LTI(client)126: got transport_rsp back from Transport layer: TransportRsp::Accepted
Sun 2023-01-22 10:48:51.000859 [TRACE] - [F<-T,R] Command Accepted
Sun 2023-01-22 10:48:51.014190 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
LTI(server)381 skipped 91: receiv

In [24]:
get_all_notification(server_fiface)

[NewRequestAction { endpoint: Endpoint(127.0.0.1:62012), action: NewRoom { room_name: "general" } }]

Sun 2023-01-22 10:48:51.247732 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:51.261963 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:51.498498 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:51.509699 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }


### Join a room

In [25]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

Sun 2023-01-22 10:48:51.731770 [TRACE] - [F<-A,C] New command: SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: JoinRoom { room_name: "general" } }
LTI(client)134 skipped 6: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, retry_interval: 250ms }], packets: [Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }] }


Accepted

Sun 2023-01-22 10:48:51.732600 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, retry_interval: 250ms }], packets: [Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }] }
Sun 2023-01-22 10:48:51.732638 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
LTI(client)135: got transport_rsp back from Transport layer: TransportRsp::Accepted
Sun 2023-01-22 10:48:51.735819 [TRACE] - [F<-T,R] Command Accepted
Sun 2023-01-22 10:48:51.758161 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:51.759261 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:6

In [26]:
get_all_notification(server_fiface)

[NewRequestAction { endpoint: Endpoint(127.0.0.1:62012), action: JoinRoom { room_name: "general" } }]

Sun 2023-01-22 10:48:51.988041 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:48:52.007339 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:48:52.008444 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:52.037899 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:52.238244 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZW

Sun 2023-01-22 10:48:54.547810 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:48:54.553975 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:48:54.578398 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:54.585568 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:54.808001 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZW

Sun 2023-01-22 10:48:57.088489 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:48:57.102794 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:48:57.118182 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:57.134542 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:57.348442 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZW

Sun 2023-01-22 10:48:59.667779 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:48:59.677985 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:59.680062 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:48:59.680071 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:48:59.918679 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZW

...

Sun 2023-01-22 10:50:21.558363 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:50:21.558382 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:50:21.559504 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:50:21.559523 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:50:21.808614 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZW

Sun 2023-01-22 10:50:24.118425 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:50:24.118444 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:50:24.134893 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:50:24.134911 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:50:24.377980 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZW

Sun 2023-01-22 10:50:26.667868 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:50:26.667886 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:50:26.681113 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:50:26.681134 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:50:26.927990 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZW

Sun 2023-01-22 10:50:29.218008 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:50:29.218026 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:50:29.233267 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:50:29.233283 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:50:29.468421 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZW

Sun 2023-01-22 10:50:31.768236 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:50:31.768256 [TRACE] - [T<-UDP] Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:50:31.781734 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: NewRoom { room_name: "general" } }
Sun 2023-01-22 10:50:31.781752 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 3, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQ1MDg5'"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:50:32.028264 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZW